In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import os
import niche
import csv
import time
out_location = "paper_webs_0624"

/Users/luva6824/jhub_venv/lib/python3.6/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
# Helper fn to generate links based on niche model parameters - same as in niche model code...
def assign_links(webb,cc,rr,nn):
    #figure out who each species eats
    S = webb.nodes()
    for x in S:
        for y in S:
            #x eats those with ny in rx around cx
            lower_bound = cc[x] - rr[x]/2
            upper_bound = cc[x] + rr[x]/2
            if lower_bound < nn[y] and nn[y] < upper_bound:
                #x eats y, add edge from y to x 
                webb.add_edge(y,x)
    return webb

In [8]:
# Link prediction - Generate new network structure with possible invaders
# 45 structures produced with invader 1 per web (scenario 1)
# 45 structures produced with invader 2 per web (scenario 2)
# 729 structures produced with invaders 1 and 2 per web (scenario 3)

webs1 = []
fif_webs_file = open('./' + out_location + '/over15_webs.txt','r')
for row in fif_webs_file:
    webs1.append(int(row))
fif_webs_file.close()

#invader 1 properties
inv_1_file = open('./' + out_location + '/invader_1.csv','r')
inv_1_reader = csv.reader(inv_1_file)
next(inv_1_reader) # get past header
inv1s = []
for row in inv_1_reader:
    inv1s.append(row)
    
#invader 2 properties
inv_2_file = open('./' + out_location + '/invader_2.csv','r')
inv_2_reader = csv.reader(inv_2_file)
next(inv_2_reader) # get past header
inv2s = []
for row in inv_2_reader:
    inv2s.append(row)

#both invader properties
inv_both_file = open('./' + out_location + '/invader_both.csv','r')
inv_both_reader = csv.reader(inv_both_file)
next(inv_both_reader) # get past header
invboths = []
for row in inv_both_reader:
    invboths.append(row)

inv_1_file.close()
inv_2_file.close()
inv_both_file.close()

for i in webs1:
        
    start = time.time()

    print("generating structures for web " + str(i))

    recipient_net,nis,ris,cis,b_curr = niche.read_web_from_file("./" + out_location + "/web_" + str(i) + "_2000")

    #output folder to save network structures shortly after invasion
    if not os.path.exists('./' + out_location + '/web_' + str(i) + '_inv'):
        os.makedirs('./' + out_location + '/web_' + str(i) + '_inv')

    #add them for invader 1
    if not os.path.exists('./' + out_location + '/web_' + str(i) + '_inv/one'):
        os.makedirs('./' + out_location + '/web_' + str(i) + '_inv/one')
    for row in inv1s:

        inv_id = int(row[0])

        nis[inv_id] = float(row[1])
        ris[inv_id] = float(row[2])
        cis[inv_id] = float(row[3])
        b_curr[inv_id] = 0.75 # always introduce with 0.75

        #generate links
        web_new = nx.DiGraph()
        web_new.add_nodes_from(recipient_net.nodes())
        web_new.add_node(inv_id) #add introduced species node
        web_new = assign_links(web_new,cis,ris,nis)

        where = './' + out_location + '/web_' + str(i) + '_inv/one/web_' + str(i) +'_inv_' + str(inv_id)
        niche.write_web_to_file(web_new,nis,ris,cis,b_curr,where)

        #reset parameter objects for next invader 
        nis.pop(inv_id)
        ris.pop(inv_id)
        cis.pop(inv_id)
        b_curr.pop(inv_id)

    print("done invader 1")

    #add them for invader 2
    if not os.path.exists('./' + out_location + '/web_' + str(i) + '_inv/two'):
        os.makedirs('./' + out_location + '/web_' + str(i) + '_inv/two')
    for row in inv2s:
        inv_id = int(row[0])

        nis[inv_id] = float(row[1])
        ris[inv_id] = float(row[2])
        cis[inv_id] = float(row[3])
        b_curr[inv_id] = 0.75

        #generate links
        web_new = nx.DiGraph()
        web_new.add_nodes_from(recipient_net.nodes())
        web_new.add_node(inv_id) #add introduced species node
        web_new = assign_links(web_new,cis,ris,nis)

        where = './' + out_location + '/web_' + str(i) + '_inv/two/web_' + str(i) +'_inv_' + str(inv_id)
        niche.write_web_to_file(web_new,nis,ris,cis,b_curr,where)

        #reset parameter objects for next invader 
        nis.pop(inv_id)
        ris.pop(inv_id)
        cis.pop(inv_id)
        b_curr.pop(inv_id)

    print("done invader 2")

    #add them for both invaders
    if not os.path.exists('./' + out_location + '/web_' + str(i) + '_inv/both'):
        os.makedirs('./' + out_location + '/web_' + str(i) + '_inv/both')
    for row in invboths:
        inv_id1 = int(row[0])
        inv_id2 = int(row[1])

        nis[inv_id1] = float(row[2])
        nis[inv_id2] = float(row[3])
        ris[inv_id1] = float(row[4])
        ris[inv_id2] = float(row[5])
        cis[inv_id1] = float(row[6])
        cis[inv_id2] = float(row[7])
        b_curr[inv_id1] = 0.75
        b_curr[inv_id2] = 0.75

        #generate links
        web_new = nx.DiGraph()
        web_new.add_nodes_from(recipient_net.nodes())
        web_new.add_node(inv_id1) #add introduced species node
        web_new.add_node(inv_id2) #add introduced species node
        web_new = assign_links(web_new,cis,ris,nis)

        where = './' + out_location + '/web_' + str(i) + '_inv/both/web_' + str(i) +'_inv_' + str(inv_id1) + '_' + str(inv_id2)
        niche.write_web_to_file(web_new,nis,ris,cis,b_curr,where)

        #reset parameter objects for next invader 
        nis.pop(inv_id1)
        ris.pop(inv_id1)
        cis.pop(inv_id1)
        b_curr.pop(inv_id1)

        nis.pop(inv_id2)
        ris.pop(inv_id2)
        cis.pop(inv_id2)
        b_curr.pop(inv_id2)

    print("done both invaders")
    end = time.time()
    print("overall time - " + str(end - start))


generating structures for web 0
done invader 1
done invader 2
done both invaders
overall time - 3.8844833374023438
generating structures for web 4
done invader 1
done invader 2
done both invaders
overall time - 3.5103037357330322
generating structures for web 8
done invader 1
done invader 2
done both invaders
overall time - 3.145132303237915
generating structures for web 10
done invader 1
done invader 2
done both invaders
overall time - 4.281385898590088
generating structures for web 13
done invader 1
done invader 2
done both invaders
overall time - 3.467247724533081
generating structures for web 14
done invader 1
done invader 2
done both invaders
overall time - 4.038255214691162
generating structures for web 16
done invader 1
done invader 2
done both invaders
overall time - 3.929121494293213
generating structures for web 18
done invader 1
done invader 2
done both invaders
overall time - 3.3471741676330566
generating structures for web 19
done invader 1
done invader 2
done both invader